In [0]:
import tensorflow as tf
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Input,SpatialDropout1D, Embedding, LSTM, Dense, merge, Convolution2D, Lambda, GRU, TimeDistributed, Reshape, Permute, Convolution1D, Masking, Bidirectional
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras import optimizers, callbacks

In [36]:
from tensorflow.python.client import device_lib
import gzip
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2232007804303425123
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15207088956788133005
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17779155790094378348
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11326753997
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5893045968978610728
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [37]:
# !git clone https://github.com/Arpita33/StrongILSData.git
# !git clone https://github.com/Arpita33/WeakILSDataset.git
!git clone https://github.com/pythonLoader/11-taxonDataset.git

fatal: destination path '11-taxonDataset' already exists and is not an empty directory.


In [0]:
import numpy as np

In [73]:
gt_strongILS_whole = np.load("11-taxonDataset/StrongILS/strong_ILS_valid_tree.npy")
gt_strongILS_taxa_removed = np.load("11-taxonDataset/StrongILS/strong_ILS_train_tree.npy")

gt_weakILS_whole = np.load("11-taxonDataset/WeakILS/weak_ILS_valid_tree.npy")
gt_weakILS_taxa_removed = np.load("11-taxonDataset/WeakILS/weak_ILS_train_tree_final.npy")

gt_total_whole = gt_weakILS_whole
gt_total_taxa_removed = gt_weakILS_taxa_removed

#gt_total_whole = np.concatenate((gt_strongILS_whole, gt_weakILS_whole), axis=0, out=None)
#gt_total_taxa_removed = np.concatenate((gt_strongILS_taxa_removed,gt_weakILS_taxa_removed),axis=0, out=None)

#gt_total_whole.shape

Train_data_X_arr = gt_total_taxa_removed
Train_data_Y_arr = gt_total_whole

print(Train_data_Y_arr.shape)

(3900, 165, 3)


In [0]:
#np.sum(Train_data_Y_arr, axis=-1).all()

In [74]:
x = Train_data_X_arr[153][:40]
# if(np.sum(Train_data_X_arr, axis=-1) == 0):
#   print("Here")
#y = Train_data_Y_arr[0][:40]

print(x)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]]


In [0]:
Train_data_X_reshaped = Train_data_X_arr.reshape(Train_data_X_arr.shape[0],Train_data_X_arr.shape[1]*Train_data_X_arr.shape[2])

Train_data_Y_reshaped = Train_data_Y_arr.reshape(Train_data_Y_arr.shape[0],Train_data_Y_arr.shape[1]*Train_data_Y_arr.shape[2])

In [76]:
print(Train_data_X_reshaped.shape)

#print(Train_data_Y_reshaped.shape)

#print(Train_data_X_reshaped[0][0:20])

#print(Train_data_Y_reshaped[0][0:20])

(3900, 495)


In [0]:
# def array_splitter(arr):
#   temp = []
#   size_of_arr = int(arr.shape[0]//3)
#   print(size_of_arr)
#   for i in range(1,size_of_arr+1):
#     t = arr[3*(i-1) : 3*i]
#     #print(t)
#     temp.append(t)
#     #print(3*(i-1), 3*i)
#   return np.asarray(temp)

In [0]:
# from keras.layers.core import Reshape

# def array_splitter(arr):
#   return arr.reshape(int(arr.shape[0]//3),3)

In [0]:
# import numpy as np
# a = np.zeros([495])
# array_splitter(a)

In [0]:
from keras.layers import Embedding, Add, concatenate, Input, Softmax
from keras import backend as K
#keras.activations.softmax(x, axis=-1)
from keras.activations import softmax


def get_model(input_size=495):
  
  #https://blog.keras.io/building-autoencoders-in-keras.html
  
  main_input = Input(batch_shape=(None, input_size), name='main_input')
  
  enc1 = Dense(int(input_size//1.5), activation='relu', use_bias=True)(main_input)
  enc2 = Dense(int(input_size//2), activation='relu', use_bias=True)(enc1)
  enc3 = Dense(int(input_size//2.5), activation='relu', use_bias=True)(enc2)
  enc4 = Dense(int(input_size//3),activation='relu', use_bias=True)(enc3)
    
  dec0 = Dense(int(input_size//2.5),activation='relu', use_bias=True)(enc4)
  dec1 = Dense(int(input_size//2), activation='relu', use_bias=True)(dec0)
  dec2 = Dense(int(input_size//1.5), activation='relu', use_bias=True)(dec1)
  dec_output = Dense(input_size, activation='sigmoid', use_bias=True)(dec2)
  
  final_decoded_layer = []
  
  # split korbi
  
  reshaped = Lambda( lambda x: K.reshape(x, (-1, int(dec_output.shape[1]//3), 3)))(dec_output)
  
  final_decoded_layer_output = Softmax(axis=-1)(reshaped)
  
  
  autoencoder = Model(main_input, final_decoded_layer_output)
  #autoencoder = Model(main_input, dec_output)
  
  ## extra
  encoder = Model(main_input, enc4)
#   encoded_input = Input(batch_shape=(None, input_size))
#   decoder_layer = autoencoder.layers[-1]
#   decoder = Model(encoded_input, decoder_layer(encoded_input))
  
  return autoencoder, encoder  #, decoder, decoder_layer, encoded_input

In [81]:
autoencoder, encoder = get_model(input_size=495)

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 495)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 330)               163680    
_________________________________________________________________
dense_18 (Dense)             (None, 247)               81757     
_________________________________________________________________
dense_19 (Dense)             (None, 198)               49104     
_________________________________________________________________
dense_20 (Dense)             (None, 165)               32835     
_________________________________________________________________
dense_21 (Dense)             (None, 198)               32868     
_________________________________________________________________
dense_22 (Dense)             (None, 247)               49153     
__________

In [0]:
# https://datascience.stackexchange.com/questions/25029/custom-loss-function-with-additional-parameter-in-keras
def custom_accuracy_function(mask):
    # mask: ekta binary mask e.g [1,0,1,0,1,1,1]
    # 0 gula missing data represent kore
    def custom_accuracy(y_true, y_predict):
        y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
        y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
        is_same = K.cast(K.equal(
            y_true_labels, y_pred_labels), 'float32')
        num_same = K.cast(K.sum(is_same * mask, axis=1), 'float32')
        lengths = K.cast(K.sum(mask, axis=1), 'float32')
        return K.mean(num_same / lengths, axis=0)
      
    return custom_accuracy
  
def custom_loss_function(mask):
    def custom_loss(y_true, y_predict):
        y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
        y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
        is_same = K.cast(K.equal(
            y_true_labels, y_pred_labels), 'float32')
        num_same = K.cast(K.sum(is_same * mask, axis=1), 'float32')
        lengths = K.cast(K.sum(mask, axis=1), 'float32')
        return K.cast(1-K.mean(num_same / lengths, axis=0), 'float32')
      
    return custom_loss

In [83]:
print(Train_data_X_arr.shape)
mask = np.sum(Train_data_X_arr, axis=-1)#y_true

(3900, 165, 3)


In [84]:
print(mask.shape)
print(np.sum(mask))
#print(mask)
#mask = np.reshape(mask, (mask.shape[0], mask.shape[1], 1))
# positions_seq = np.repeat([positions_seq],traindata.shape[0], axis=0)
# print(positions_seq.shape)
#print(mask.shape)
#mask = np.repeat(mask, 3, axis=-1)
#mask.shape

(3900, 165)
466049.0


In [0]:
# Train_data_X_with_mask = np.asarray([Train_data_X_arr, mask])
# Train_data_X_with_mask = [Train_data_X_arr, mask]
# Train_data_X_with_mask = np.concatenate((Train_data_X_arr, mask), axis=-1)

In [0]:
#Train_data_X_with_mask.shape

In [87]:
"""def custom_accuracy_function(y_true_with_mask, y_predict):
    y_true_with_mask.shape
    y_true = y_true_with_mask[:, :3]
    mask = y_true_with_mask[:, 3]
    y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
    y_true_labels = K.argmax(y_true, axis=2)
    y_true_labels = K.cast(y_true_labels, 'float32')
    is_same = K.cast(K.equal(
        y_true_labels, y_pred_labels), 'float32')
    num_same = K.sum(is_same * mask, axis=1)
    lengths = K.sum(mask, axis=1)
    return K.mean(num_same / lengths, axis=0)"""

"def custom_accuracy_function(y_true_with_mask, y_predict):\n    y_true_with_mask.shape\n    y_true = y_true_with_mask[:, :3]\n    mask = y_true_with_mask[:, 3]\n    y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')\n    y_true_labels = K.argmax(y_true, axis=2)\n    y_true_labels = K.cast(y_true_labels, 'float32')\n    is_same = K.cast(K.equal(\n        y_true_labels, y_pred_labels), 'float32')\n    num_same = K.sum(is_same * mask, axis=1)\n    lengths = K.sum(mask, axis=1)\n    return K.mean(num_same / lengths, axis=0)"

In [0]:
best_model_file = './ae_for_imputation'
checkpoint = callbacks.ModelCheckpoint(best_model_file, monitor='binary_crossentropy', verbose=1, save_best_only=True, mode='min')

In [89]:
print(Train_data_X_reshaped.shape)

input_size = Train_data_X_reshaped.shape[0]
print(input_size)

(3900, 495)
3900


In [0]:
# https://datascience.stackexchange.com/questions/25029/custom-loss-function-with-additional-parameter-in-keras
#autoencoder.compile(optimizer='adam', loss)

#adam = Adam(lr=0.0001)

autoencoder.compile(optimizer='adam',
              loss='binary_crossentropy', #https://github.com/LucaAngioloni/ProteinSecondaryStructure-CNN/blob/master/dataset.py
              sample_weight_mode='temporal',
              metrics=[custom_accuracy_function(mask=mask)])

In [91]:
autoencoder.fit(x=Train_data_X_reshaped, y=Train_data_X_arr,batch_size=input_size,
        epochs=1000,
        shuffle=True,
        validation_split = 0.0,
        sample_weight=mask,
        verbose = 1)

Epoch 1/1000
3900/3900 [==============================] - 1s 205us/step - loss: 0.6363 - custom_accuracy: 0.3462
Epoch 2/1000
3900/3900 [==============================] - 0s 18us/step - loss: 0.6327 - custom_accuracy: 0.4952
Epoch 3/1000
3900/3900 [==============================] - 0s 19us/step - loss: 0.6234 - custom_accuracy: 0.5971
Epoch 4/1000
3900/3900 [==============================] - 0s 19us/step - loss: 0.6030 - custom_accuracy: 0.6798
Epoch 5/1000
3900/3900 [==============================] - 0s 18us/step - loss: 0.5655 - custom_accuracy: 0.7459
Epoch 6/1000
3900/3900 [==============================] - 0s 18us/step - loss: 0.5106 - custom_accuracy: 0.7948
Epoch 7/1000
3900/3900 [==============================] - 0s 19us/step - loss: 0.4514 - custom_accuracy: 0.8267
Epoch 8/1000
3900/3900 [==============================] - 0s 19us/step - loss: 0.4066 - custom_accuracy: 0.8483
Epoch 9/1000
3900/3900 [==============================] - 0s 18us/step - loss: 0.3795 - custom_accuracy

In [0]:
imputed_position_mask = 1 - mask

In [93]:
imputed_position_mask.shape

(3900, 165)

In [0]:
y_pred = autoencoder.predict(Train_data_X_reshaped)

In [0]:
def direct_similarity_check_for_imputation(sample_no,Y_tree, Y_hat_tree, imputed_position_mask_for_tree, verbose=0):
#   ck = 0
  mismatch_count = 0
  print("checking tree ->" + str(sample_no))
  #print(np.sum(imputed_position_mask_for_tree))
  for i in range(len(Y_hat_tree)):
    
    if (imputed_position_mask_for_tree[i]==1):
      if(verbose == 0):
        print(i, '>', Y_hat_tree[i], Y_tree[i], imputed_position_mask_for_tree[i])
      if(np.argmax(Y_hat_tree[i]) != np.argmax(Y_tree[i])):
        #print('mismatch at:', i, Y_hat_tree[i], Y_tree[i])
        mismatch_count += 1
    else:
      if(verbose == 0):
        print(i, '>', Y_hat_tree[i], Y_tree[i], imputed_position_mask_for_tree[i])
  #       ck = 1
  #   if ck == 0:
  #     print('>>Fully matched.')
  #     return True
  #   return False
  return mismatch_count

In [110]:
mismatch_count = 0
temp = 0
v_count = 0
for test_sample in range(input_size):
  #print('sample:', test_sample)
  temp = direct_similarity_check_for_imputation(test_sample,Train_data_Y_arr[test_sample], y_pred[test_sample], imputed_position_mask_for_tree=imputed_position_mask[test_sample], verbose=v_count)
  v_count += 1
  mismatch_count += temp
  #print("number of mismatch in this tree -> " + str(temp))
  
print(mismatch_count)

temp2 = np.sum(imputed_position_mask)
print(temp2)  
#acc = (1-mismatch_count/(np.size(Train_data_Y_arr,1)*np.size(Train_data_Y_arr,0)))
acc = (1-mismatch_count/temp2)
percent_acc =  acc*100
print(str(percent_acc) + "%")


checking tree ->0
109.0
0 > [0.21194156 0.21194156 0.57611686] [0. 0. 1.] 1.0
1 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
2 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
3 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
4 > [0.5761169  0.21194157 0.21194157] [1. 0. 0.] 1.0
5 > [0.5761169  0.21194157 0.21194157] [1. 0. 0.] 1.0
6 > [0.5761169  0.21194157 0.21194157] [1. 0. 0.] 1.0
7 > [0.5761169  0.21194157 0.21194157] [1. 0. 0.] 1.0
8 > [0.5761169  0.21194157 0.21194157] [1. 0. 0.] 1.0
9 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
10 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
11 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
12 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
13 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
14 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
15 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
16 > [0.21194157 0.5761169  0.21194157] [0. 1. 0.] 1.0
17 > [0.21194156 0.21194156 0.57611686] [0. 0. 1.] 

In [97]:
print(mask.shape)
print(mask.shape[0]*mask.shape[1])
temp2 = np.sum(imputed_position_mask)  
print(temp2)
# mask_temp1 = mask[:int(mask.shape[0]/3)]
# print(mask_temp1.shape)

# autoencoder.train_on_batch(x=Train_data_X_reshaped, y=Train_data_X_arr,sample_weight=mask_temp1)

(3900, 165)
643500
177451.0


In [98]:
test_sample = 13
cnt=direct_similarity_check_for_imputation(test_sample,Train_data_Y_arr[test_sample], y_pred[test_sample], imputed_position_mask_for_tree=imputed_position_mask[test_sample], verbose=1)
print(cnt)

0


In [99]:
np.sum(imputed_position_mask[test_sample])

0.0